# ***Se importa lo necesario***

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import json
from scipy import stats


In [ ]:
#acá se extraen las estructuras centro-periferia construidas en el codigo numero 1
año_inicial = 1999
año_final = 2021
import json
estructuras = {}
for año in range(año_inicial, año_final + 1):
  with open('/gdrive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Codigos/estructuras_'+ str(año) +'.json', "r") as f:
    estructura = json.load(f)
    estructuras[str(año)] = estructura

In [ ]:
#se crea un array con los años y se crea una ruta hacía donde están los datos procesados 
home_data = "/gdrive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Datos/Processed_Data/BACI_HS92_V202301"
years = np.linspace(1999,2021,2021-1999 +1)

In [ ]:
# Acá se define la clase economic entropys, con la cual se calculan todas las balanzas a excepción de la balanza de poder
class Economic_Balances:
    def __init__(self, data, center, periphery):
        self.data = data
        self.products = data['Product category_code'].unique()
        self.countrys = pd.concat([data['Importer'], data['Exporter']]).unique()
        self.center = center
        self.periphery = periphery

    def import_joint_prob(self, importers=None): 
        # Si no se proveen importadores, se utilizan todos los paises
        if importers is None:
            importers = self.countrys

        # se filtran los datos acorde a los importadores
        data_importers = self.data[self.data['Importer'].isin(importers)]
        data_grouped_i = data_importers.groupby(['Product category_code', 'Exporter'])['Value'].sum().reset_index()

        # Filtra la data por exportadores centro y periferia
        df_periphery = data_grouped_i[data_grouped_i['Exporter'].isin(self.periphery)]
        df_center = data_grouped_i[data_grouped_i['Exporter'].isin(self.center)]

        # Se agrupa por categoria de producto y se suma el valor
        result_periphery = df_periphery.groupby('Product category_code')['Value'].sum().reset_index()
        result_center = df_center.groupby('Product category_code')['Value'].sum().reset_index()

        # Concatenar los resultados, asignar los valores de Exportador correspondientes y ordenar por código_categoría de Producto
        result = pd.concat([result_periphery.assign(Exporter='periphery'), result_center.assign(Exporter='center')])
        result = result.sort_values('Product category_code').reset_index(drop=True)

        # Crear una tabla pivot y convertirla en un array
        probconjunta = result.pivot_table(values='Value', index='Exporter', columns='Product category_code').fillna(0).values
        # Normalizar las probabilidades
        probconjunta /= np.sum(probconjunta)
        return probconjunta

    def trade_distr(self, importers = None, exporters =None):
        # Si no se proveen exportadores o importadores, se usa todo el mundo
        if importers is None:
            importers = self.countrys
        if exporters is None:
            exporters = self.countrys
        # Filtrar la data dados los importadores y exportadores
        data_filtered = self.data[(self.data['Importer'].isin(importers)) & (self.data['Exporter'].isin(exporters))]
        return data_filtered

    def trade_prob(self,importers = None, exporters = None):
        data_filtered = self.trade_distr( importers = importers, exporters = exporters)
        # Se agrupa por categoria de producto y se suma el valor
        import_distribution = data_filtered.groupby('Product category_code')['Value'].sum().values
        # se calculan la distribucion de probabilidad de importar una categoria de producto
        return import_distribution / np.sum(import_distribution)

    def s_trade(self, importers=None, exporters=None): #calcula entropía de la distribución
        import_prob = self.trade_prob(importers = importers, exporters = exporters)
        return stats.entropy(import_prob, base=2)

    def bal_money(self, importers = None, exporters = None): #calcula balanza de entropia
      df = self.data.copy()
      trade_import = self.trade_distr(importers = exporters, exporters = importers)
      trade_export = self.trade_distr(importers = importers, exporters = exporters)
      net_imports = trade_import['Value'].sum()
      net_exports = trade_export['Value'].sum()
      return net_exports - net_imports

    def bal_ton(self, importers = None, exporters = None): #calcula balanza de toneladas
      df = self.data.copy()
      trade_import = self.trade_distr(importers = exporters, exporters = importers)
      trade_export = self.trade_distr(importers = importers, exporters = exporters)
      net_imports = trade_import['Quantity'].sum()
      net_exports = trade_export['Quantity'].sum()
      return net_exports - net_imports
      

In [ ]:
año_inicial = 1999
año_final = 2021

for año in range(año_inicial, año_final + 1):
  df_resultados = pd.DataFrame()
  with open('/gdrive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Codigos/estructuras_'+ str(año) +'.json', "r") as f:
    estructuras = json.load(f)

  dic_jerarquias_BTP = {}
  dic_bal_entropias = {} 
  dic_bal_dinero = {} 
  dic_bal_ton = {} 
  dic_bal_uni = {}

  data = pd.read_csv(home_data + '/Y'+ str(año) + '/' + 'Enriched_BACI_HS92_Y' + str(año) + '_V202301.csv')
  medidas = Economic_Balances(data,center,periphery)

  for estructura in estructuras:
    nodos = estructuras[estructura]['nodos']
    dic_jerarquias_BTP[estructura] = estructuras[estructura]['BTP']
    dic_bal_entropias[estructura] = medidas.s_trade(exporters = nodos) - medidas.s_trade(importers = nodos)
    dic_bal_dinero[estructura] = medidas.bal_money(exporters = nodos)
    dic_bal_ton[estructura] = medidas.bal_ton(exporters = nodos)
    dic_bal_uni[estructura] = medidas.unicity_bal(exporters = nodos)

    df_fila = pd.DataFrame({'año' : [año],  'nivel' : [estructura], 'BTP' : dic_jerarquias_BTP[estructura], 'BS': dic_bal_entropias[estructura],
                            'BC': dic_bal_dinero[estructura],'BTON' : dic_bal_ton[estructura], 'BUNI' : dic_bal_uni[estructura]
                            })
    df_resultados = pd.concat([df_resultados, df_fila])

  df_resultados.to_csv('/gdrive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Resultados/' + str(año)+ 'resultados_nivel_BTP.csv', index=False)
  print("guardado el del año " + str(año))